# Panel widgets

In [27]:
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
import numpy as np
import io
import pandas as pd
import panel as pn
from typing import Dict, List
import seaborn as sns


plt.ion()

In [28]:
pn.extension()
upload_local = pn.widgets.FileInput(
    multiple=True,
    accept='.tsv',
)

def bytes_to_df(data:bytes, sep:str = "\t") -> Dict[str, pd.DataFrame]:
    """
    Convert a dictionary of bytes to a dictionary of DataFrames.

    Args:
        data (Dict[str, bytes]): A dictionary where keys are filenames and values are byte strings.
        sep (str): The separator used in the data files. Default is tab ("\t").

    Returns:
        Dict[str, pd.DataFrame]: A dictionary where keys are filenames and values are DataFrames.
    """
    df = pd.read_csv(
        io.StringIO(
            str(
                data,
                'utf-8',
            )
        ),
        sep=sep,
    )
    return df

def bgc_tables_dict(tsv_data: list, tsv_names: list):
    """
    Combine multiple BGC tables into a single DataFrame.

    Args:
        tvs_files (list): List of file paths to the TVS files.

    Returns:
        pd.DataFrame: Combined DataFrame with all BGC data.
    """
    tables_dict = {}
    for i, data in enumerate(tsv_data):
        try:
            df = bytes_to_df(data)
            tables_dict[tsv_names[i]] = df
        except Exception as e:
            continue
    return tables_dict


def mpl_bgcs_violin(df):
    plot = plt.figure(figsize=(10, 6), facecolor=(0, 0, 0, 0))
    ax = plot.add_subplot(111)

    df['type'] = df['type'].fillna('Unknown')

    sns.swarmplot(
        data=df, x="type", y="average_p",
        hue="max_p", size=7,
        palette="coolwarm",
        ax=ax)
    sns.violinplot(
        data=df, x="type", y="average_p",
        inner=None, fill=False,
        color="black",
        ax=ax,
    )
    ax.set_ylim(0, 1)
    
    ax.set_title(f"Probabilities of identified BGCs by type")
    ax.set_xlabel("BGC type")
    ax.set_ylabel("Average probability")
    # ax = cut_xaxis_labels(ax, 15)

    plt.tight_layout()
    plt.close(plot)
    return plot

In [29]:
upload_local

BokehModel(combine_events=True, render_bundle={'docs_json': {'94b303cf-bd3b-436e-ba8d-258d760391cb': {'version…

In [30]:
tables = bgc_tables_dict(upload_local.value, upload_local.filename)

In [31]:
tables.keys()

dict_keys(['HCFCYDSX5.UDI129_final.contigs.clusters.tsv', 'HCFCYDSX5.UDI130_final.contigs.clusters.tsv'])

In [32]:
df = tables['HCFCYDSX5.UDI129_final.contigs.clusters.tsv']

In [33]:
df.head()

,sequence_id,cluster_id,start,end,average_p,max_p,type,alkaloid_probability,nrp_probability,polyketide_probability,ripp_probability,saccharide_probability,terpene_probability,proteins,domains
0,k141_1048287,k141_1048287_cluster_1,129,3952,0.957671,0.963429,Terpene,0.033073,0.132545,0.156133,0.122384,0.01,0.511946,k141_1048287_1;k141_1048287_2;k141_1048287_3,PF00432;PF01839;PF01839;PF01839;PF13243;PF1324...
1,k141_1056062,k141_1056062_cluster_1,2674,6382,0.999430,0.999944,Unknown,0.077437,0.121138,0.131008,0.113054,0.27,0.192597,k141_1056062_4;k141_1056062_5;k141_1056062_6,PF00078;PF00098;PF17917;PF17919;PF17921
2,k141_1065989,k141_1065989_cluster_1,2,2703,0.928524,0.932066,NRP,0.028140,0.705660,0.123313,0.028834,0.00,0.027847,k141_1065989_1;k141_1065989_2;k141_1065989_3,PF00550;PF00668;PF03781;PF03781
3,k141_1067118,k141_1067118_cluster_1,66,11014,0.940587,0.999159,Unknown,0.108911,0.074195,0.124557,0.074153,0.03,0.222321,k141_1067118_1;k141_1067118_2;k141_1067118_3;k...,PF00067;PF00494;PF00501;PF00512;PF01209;PF0122...
4,k141_1113628,k141_1113628_cluster_1,2,4104,0.952922,0.996064,Unknown,0.030699,0.204059,0.297264,0.171291,0.01,0.215222,k141_1113628_1;k141_1113628_2;k141_1113628_3;k...,PF00027;PF01636;PF17914


In [34]:
bgc_types = pn.pane.Matplotlib(
    height=600,
    name="BGC types",
    )

In [35]:
bgc_types

BokehModel(combine_events=True, render_bundle={'docs_json': {'53ccd87f-383a-4107-ab49-540c934c4039': {'version…

In [36]:
bgc_types.object = mpl_bgcs_violin(df)

/home/david-palecek/miniforge3/envs/jupyterbook-template/lib/python3.13/site-packages/seaborn/categorical.py:3399: UserWarning: 14.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/home/david-palecek/miniforge3/envs/jupyterbook-template/lib/python3.13/site-packages/seaborn/categorical.py:3399: UserWarning: 17.4% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


### Leafmap in ipynb

In [3]:
from ipyleaflet import Map, Marker

center = (48.866667, 2.333333)
map = Map(center=center, zoom=12)

# Add a draggable marker to the map
# Dragging the marker updates the marker.location value in Python
marker = Marker(location=center, draggable=True)
map.add_control(marker)

display(map)

Map(center=[48.866667, 2.333333], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [4]:
map

Map(bottom=360895.0, center=[48.866747381009176, 2.3333930969238286], controls=(ZoomControl(options=['position…

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)